# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Summary" data-toc-modified-id="Summary-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Summary</a></div><div class="lev1 toc-item"><a href="#Version-Control" data-toc-modified-id="Version-Control-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Version Control</a></div><div class="lev1 toc-item"><a href="#Change-Log" data-toc-modified-id="Change-Log-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Change Log</a></div><div class="lev1 toc-item"><a href="#Setup" data-toc-modified-id="Setup-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Setup</a></div><div class="lev1 toc-item"><a href="#FundamentalList()" data-toc-modified-id="FundamentalList()-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>FundamentalList()</a></div><div class="lev2 toc-item"><a href="#Web-service-call" data-toc-modified-id="Web-service-call-51"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Web service call</a></div><div class="lev3 toc-item"><a href="#Gather-elements" data-toc-modified-id="Gather-elements-511"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Gather elements</a></div><div class="lev3 toc-item"><a href="#Get-data" data-toc-modified-id="Get-data-512"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>Get data</a></div><div class="lev3 toc-item"><a href="#Save-to-file" data-toc-modified-id="Save-to-file-513"><span class="toc-item-num">5.1.3&nbsp;&nbsp;</span>Save to file</a></div><div class="lev3 toc-item"><a href="#Data-inspection" data-toc-modified-id="Data-inspection-514"><span class="toc-item-num">5.1.4&nbsp;&nbsp;</span>Data inspection</a></div><div class="lev2 toc-item"><a href="#Helper-function" data-toc-modified-id="Helper-function-52"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Helper function</a></div><div class="lev3 toc-item"><a href="#Usage" data-toc-modified-id="Usage-521"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Usage</a></div><div class="lev2 toc-item"><a href="#Client-function" data-toc-modified-id="Client-function-53"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Client function</a></div>

# Summary

Part of the blog series related to making web service calls to Eoddata.com. Overview of the web service can be found [here](http://ws.eoddata.com/data.asmx).

 * ** View the master post of this series to build a secure credentials file.** It is used in all posts related to this series.
 * Download this blog post as a [jupyter notebook](https://adriantorrie.github.io/downloads/notebooks/eoddata/eoddata_web_service_calls_fundamental_list.ipynb)
 * Download the [class definition file](https://adriantorrie.github.io/downloads/code/eoddata.py) for an easy to use client, which is demonstrated below
 * This post covers the `FundamentalList` call: http://ws.eoddata.com/data.asmx?op=FundamentalList

# Version Control

In [1]:
%run ../../code/version_check.py

Python: 3.5.3 |Continuum Analytics, Inc.| (default, Feb 22 2017, 21:13:27) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]

matplotlib:	2.0.0
numpy:		1.12.0
pandas:		0.19.2
quandl:		3.0.1
sklearn:	0.18.1
scipy:		0.18.1
statsmodels:	0.8.0
tensorflow:	1.0.1


# Change Log

    Date Created: 2017-04-02
    
    Date of Change    Change Notes
    --------------    ----------------------------------------------------------------
    2017-04-02        Initial draft

# Setup

In [2]:
%run ../../code/eoddata.py

import pandas as pd
import requests as r

ws = 'http://ws.eoddata.com/data.asmx'
ns='http://ws.eoddata.com/Data'

with (Client()) as eoddata:
    token = eoddata.get_token()

In [3]:
exchange_code = 'NASDAQ'

# FundamentalList()

## Web service call

In [4]:
session = r.Session()

call = 'FundamentalList'
kwargs = {'Token': token, 'Exchange': exchange_code}
pattern = './/{%s}FUNDAMENTAL'

url = '/'.join((ws, call))
response = session.get(url, params=kwargs, stream=True)

if response.status_code == 200:
    root = etree.parse(response.raw).getroot()
    
session.close()

### Gather elements

In [5]:
elements = root.findall(pattern %(ns))

### Get data

In [6]:
fundamentals = sorted((element.get('Symbol'), element.get('Name')) for element in elements)
fundamentals[:10]

[('AAAP', 'Advanced Accele. Ads'),
 ('AAL', 'American Airlines Gp'),
 ('AAME', 'Atlantic Amer Cp'),
 ('AAOI', 'Applied Optoelect Cmn'),
 ('AAON', 'Aaon Inc'),
 ('AAPC', 'Atlantic Alliance Or'),
 ('AAPL', 'Apple Inc'),
 ('AAWW', 'Atlas Air Ww'),
 ('AAXJ', 'Ishares MSCI Aca X-Japan'),
 ('ABAC', 'Aoxin Tianli Grp')]

### Save to file

In [7]:
with open('../../data/nasdaq_fundamentals.csv', 'w') as f:
    for element in elements:
        f.write('"%s"\n' % '","'.join(element.attrib.values()))

### Data inspection

In [8]:
for item in root.items():
    print (item)

('Source', 'Data.FundamentalList')
('Message', 'Success')
('Date', '2017-04-02T07:42:54.018-05:00')


In [9]:
for element in root.iter():
    print(element.attrib)

{'Source': 'Data.FundamentalList', 'Message': 'Success', 'Date': '2017-04-02T07:42:54.018-05:00'}
{}
{'NTA': '0', 'Yield': '14.8', 'EBITDA': '-8080000', 'Industry': '', 'PtS': '15.39', 'DateTime': '2017-03-31T00:00:00', 'EPS': '-0.67', 'Dividend': '0', 'Name': 'Advanced Accele. Ads', 'Description': 'Advanced Accele. Ads', 'DivYield': '0', 'MarketCap': '1810000000', 'DividendDate': '0001-01-01T00:00:00', 'Sector': '', 'PtB': '5.62', 'PE': '0', 'PEG': '0', 'Shares': '45408930', 'ImputationCredits': '0', 'Symbol': 'AAAP', 'DPS': '0'}
{'NTA': '0', 'Yield': '21.94', 'EBITDA': '7830000000', 'Industry': '', 'PtS': '0.53', 'DateTime': '2017-03-31T00:00:00', 'EPS': '4.81', 'Dividend': '0', 'Name': 'American Airlines Gp', 'Description': 'American Airlines Gp', 'DivYield': '0.94', 'MarketCap': '21330000000', 'DividendDate': '0001-01-01T00:00:00', 'Sector': '', 'PtB': '5.7', 'PE': '8.79', 'PEG': '4.22', 'Shares': '504255300', 'ImputationCredits': '0', 'Symbol': 'AAL', 'DPS': '0.4'}
{'NTA': '0', 'Y

## Helper function

In [10]:
def FundamentalList(session, token, exchange_code):
    call = 'FundamentalList'
    kwargs = {'Token': token, 'Exchange': exchange_code}
    pattern = ".//{%s}FUNDAMENTAL"

    url = '/'.join((ws, call))
    response = session.get(url, params=kwargs, stream=True)

    if response.status_code == 200:
        root = etree.parse(response.raw).getroot()
        
    return sorted((element.get('Symbol'), element.get('Name')) for element in elements)

### Usage

In [11]:
session = r.session()
fundamentals = FundamentalList(session, token, exchange_code)
session.close()

fundamentals[:10]

[('AAAP', 'Advanced Accele. Ads'),
 ('AAL', 'American Airlines Gp'),
 ('AAME', 'Atlantic Amer Cp'),
 ('AAOI', 'Applied Optoelect Cmn'),
 ('AAON', 'Aaon Inc'),
 ('AAPC', 'Atlantic Alliance Or'),
 ('AAPL', 'Apple Inc'),
 ('AAWW', 'Atlas Air Ww'),
 ('AAXJ', 'Ishares MSCI Aca X-Japan'),
 ('ABAC', 'Aoxin Tianli Grp')]

## Client function

In [12]:
# pandas dataframe is returned
df = eoddata.fundamental_list(exchange_code)

df.head()

,Symbol,Name,Description,DateTime,Industry,Sector,Shares,MarketCap,PE,EPS,...,DivYield,Dividend,DividendDate,DPS,ImputationCredits,EBITDA,PEG,PtS,PtB,Yield
0,AAAP,Advanced Accele. Ads,Advanced Accele. Ads,2017-03-31T00:00:00,,,45408930,1810000000,0,-0.67,...,0,0,0001-01-01T00:00:00,0,0,-8080000,0,15.39,5.62,14.8
1,AAL,American Airlines Gp,American Airlines Gp,2017-03-31T00:00:00,,,504255300,21330000000,8.79,4.81,...,0.94,0,0001-01-01T00:00:00,0.4,0,7830000000,4.22,0.53,5.7,21.94
2,AAME,Atlantic Amer Cp,Atlantic Amer Cp,2017-03-31T00:00:00,Life Insurance,Finance,20422500,81690000,36.7,0.11,...,0.54,0,0001-01-01T00:00:00,0.02,0,6290000,0,0.49,0.78,-4.31
3,AAOI,Applied Optoelect Cmn,Applied Optoelect Cmn,2017-03-31T00:00:00,,,18720470,1030000000,31.26,1.76,...,0,0,0001-01-01T00:00:00,0,0,37210000,0.81,3.69,4.17,401.34
4,AAON,Aaon Inc,Aaon Inc,2017-03-31T00:00:00,Industrial Machinery/Components,Capital Goods,52616690,1860000000,35.35,1,...,0.75,0,0001-01-01T00:00:00,0.26,0,92980000,2.67,4.83,9.01,33.3
